# Pythonを用いた競馬予測

## インポート

In [1]:
import re  # 正規表現
import time
from urllib.request import Request, urlopen

import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

## データ収集
- カレンダーのページから開催日一覧を取得（スクレイピング）
    1. 2023年1月：https://race.netkeiba.com/top/calendar.html?year=2023&month=1
- 開催ページからレースid一覧を取得
    1. 2023年1月5日開催：https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105
- レース結果ページからレース結果テーブル一覧を取得

In [3]:
from urllib.request import Request, urlopen

url = "https://db.netkeiba.com/race/202306050811/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()
print(html)

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xcd\xad\xc7\xcf\xb5\xad\xc7\xb0\xa1\xc32023\xc7\xaf12\xb7\xee24\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1

In [11]:
import pandas as pd

pd.read_html(html)

NameError: name 'html' is not defined

In [5]:
pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,3,5,ドウデュース,牡4,58,武豊,2:30.9,NaN,5.2,2,506(+2),[西] 友道康夫
1,2,8,16,スターズオンアース,牝4,56,ルメール,2:31.0,1/2,8.6,7,494(-2),[東] 高柳瑞樹
2,3,2,4,タイトルホルダー,牡5,58,横山和生,2:31.2,1,8.3,6,474(-2),[東] 栗田徹
3,4,5,10,ジャスティンパレス,牡4,58,横山武史,2:31.2,アタマ,3.6,1,474(+6),[西] 杉山晴紀
4,5,1,2,シャフリヤール,牡5,58,松山弘平,2:31.2,クビ,44.6,8,454(0),[西] 藤原英昭
5,6,7,13,タスティエーラ,牡3,56,ムーア,2:31.5,1.1/2,7.1,5,498(+18),[東] 堀宣行
6,7,6,12,ウインマリリン,牝6,56,モリス,2:31.6,3/4,171.9,14,476(0),[東] 手塚貴久
7,8,1,1,ソールオリエンス,牡3,56,川田将雅,2:31.6,アタマ,6.5,4,466(+2),[東] 手塚貴久
8,9,6,11,ハーパー,牝3,54,岩田望来,2:31.6,クビ,93.2,11,474(-10),[西] 友道康夫
9,10,2,3,ホウオウエミーズ,牝6,56,田辺裕信,2:31.7,クビ,193.2,16,452(+2),[東] 池上昌和


In [6]:
pd.read_html(html)[1]

,0,1,2,3
0,単勝,5,520,2
1,複勝,5 16 4,230 240 330,3 5 7
2,枠連,3 - 8,980,2
3,馬連,5 - 16,2730,15


In [7]:
pd.read_html(html)[2]

,0,1,2,3
0,ワイド,5 - 16 4 - 5 4 - 16,"930 1,210 1,720",13 17 21
1,馬単,5 → 16,4380,23
2,三連複,4 - 5 - 16,8050,28
3,三連単,5 → 16 → 4,42110,159


### 開催日一覧を取得する

In [50]:
url = "https://race.netkeiba.com/top/calendar.html?year=2023&month=1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()
html

HTTPError: HTTP Error 400: Bad Request

In [9]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html)
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_common_race_schedule (d) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>開催日程 | レース情報(JRA) - netkeiba</title>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,開催日程,スケジュール,開催日,netkeiba,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba_logo_keiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="開催日程 | レース情報(JRA) - netkeiba" property="og:title"/>
<meta content="https://race.netkeiba.com/top/calendar.html" property="og:url"/>
<meta content="中央競馬（JRA）開催レースの開催日程ページです。中央競馬（JRA）開催レースの出馬表や最

In [10]:
a = soup.find("table", class_="Calendar_Table").find("a")
a

<a href="../top/race_list.html?kaisai_date=20230105" target="_parent" tile="">
<div class="RaceKaisaiBox HaveData">
<p><span class="Day">5</span></p>
<p><span class="JyoName">中山</span><span class="JName">中山金杯</span></p>
<p><span class="JyoName">中京</span><span class="JName">京都金杯</span></p>
</div><!-- /. RaceKaisaiBox-->
</a>

In [11]:
a["href"]

'../top/race_list.html?kaisai_date=20230105'

In [12]:
import re # 正規表現

re.findall(r"kaisai_date=(\d{8})", a["href"])[0] # \d{8}：8桁の数字（\dは数字）()を付けるとその部分だけ取り出される

'20230105'

In [13]:
a_list = soup.find("table", class_="Calendar_Table").findAll("a")

In [14]:
kaisai_date_list = []
for a in a_list:
    kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
    kaisai_date_list.append(kaisai_date)

kaisai_date_list

['20230105',
 '20230107',
 '20230108',
 '20230109',
 '20230114',
 '20230115',
 '20230121',
 '20230122',
 '20230128',
 '20230129']

In [15]:
import time
from tqdm.notebook import tqdm

def scrape_kaisai_date(from_, to_):
    
    kaisai_date_list = []
    
    for date in tqdm(pd.date_range(from_, to_, freq="MS")):
        year = date.year
        month = date.month
        url = f"https://race.netkeiba.com/top/calendar.html?year={year}&month={month}"
        
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
        req = Request(url, headers=headers)
        html = urlopen(req).read()
        
        time.sleep(1)
        
        soup = BeautifulSoup(html, 'html.parser')
        
        a_list = soup.find("table", class_="Calendar_Table").find_all("a")
        
        for a in a_list:
            kaisai_date = re.findall(r"kaisai_date=(\d{8})", a["href"])[0]
            kaisai_date_list.append(kaisai_date)
        
    return kaisai_date_list

In [16]:
scrape_kaisai_date("2023-01", "2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 開催ページからrace_idを取得

In [54]:
url = "https://race.netkeiba.com/top/race_list.html?kaisai_date=20230105"

In [55]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [56]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
req = Request(url, headers=headers)
html = urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')


In [57]:
soup.find("div", class_="RaceList_Box") # 動的jsで構成されている場合はBeautifulSoupでは要素を取得できない.

取得できないので、ChromeDriverを使う

In [58]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

driver_path = ChromeDriverManager().install()
driver_path


'C:\\Users\\go200\\.wdm\\drivers\\chromedriver\\win64\\131.0.6778.204\\chromedriver-win32/chromedriver.exe'

In [59]:
driver = webdriver.Chrome(service=Service(driver_path))

In [60]:
driver.get(url)

In [61]:
from selenium.webdriver.common.by import By
li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
li_list

[<selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.67")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.68")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.69")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.70")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.71")>,
 <selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a8

In [62]:
li = li_list[0]
li

<selenium.webdriver.remote.webelement.WebElement (session="da116f25db9c85eaf71a027a870709fa", element="f.26FC9E413F79EDE3217770B2350C2599.d.724BD1EE013016FA85D16A03D639E795.e.67")>

In [63]:
href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
href

'https://race.netkeiba.com/race/result.html?race_id=202306010101&rf=race_list'

In [64]:
import re

re.findall(r"race_id=(\d{12})", href)[0]

'202306010101'

In [65]:
race_id_list = []

for li in li_list:
    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
    race_id = re.findall(r"race_id=(\d{12})", href)[0]
    race_id_list.append(race_id)

In [66]:
len(race_id_list)

24

In [67]:
driver.quit()

## 関数化

In [68]:
import scraping

In [69]:
kaisai_date_list = scraping.scrape_kaisai_date(from_="2023-01", to_="2023-12")

  0%|          | 0/12 [00:00<?, ?it/s]

In [70]:
import time
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm
import traceback

def scrape_race_id_list(kaisai_date_list: list[str]):
    options = Options()
    options.add_argument("--headless") # 処理軽量化のためにバックグラウンドで実行
    driver_path = ChromeDriverManager().install()
    race_id_list = []
    
    # for文終了時にwith構文自動的にdriverがquitする.
    with webdriver.Chrome(service=Service(driver_path),options=options) as driver:
        for kaisai_date in tqdm(kaisai_date_list):
            url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={kaisai_date}"
            try:
                driver.get(url)
                time.sleep(1)
                li_list = driver.find_elements(By.CLASS_NAME, "RaceList_DataItem")
                for li in li_list:
                    href = li.find_element(By.TAG_NAME, "a").get_attribute("href")
                    race_id = re.findall(r"race_id=(\d{12})", href)[0]
                    race_id_list.append(race_id)
            except:
                print(f"stopped at {url}")
                print(traceback.format_exc()) # エラー把握
                break
    return race_id_list

In [71]:
race_id_list = scrape_race_id_list(kaisai_date_list)

  0%|          | 0/107 [00:00<?, ?it/s]

stopped at https://race.netkeiba.com/top/race_list.html?kaisai_date=20230107
Traceback (most recent call last):
  File "C:\Users\go200\AppData\Local\Temp\ipykernel_30360\893257457.py", line 18, in scrape_race_id_list
    time.sleep(1)
KeyboardInterrupt



In [72]:
race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112']

### スクリプトのチェック

In [35]:
import scraping
%load_ext autoreload

In [36]:
%autoreload

In [ ]:
race_id_list = scraping.scrape_race_id_list(kaisai_date_list[:10])

notebookにおけるモジュールのリロード

In [73]:
race_id_list

['202306010101',
 '202306010102',
 '202306010103',
 '202306010104',
 '202306010105',
 '202306010106',
 '202306010107',
 '202306010108',
 '202306010109',
 '202306010110',
 '202306010111',
 '202306010112',
 '202307010101',
 '202307010102',
 '202307010103',
 '202307010104',
 '202307010105',
 '202307010106',
 '202307010107',
 '202307010108',
 '202307010109',
 '202307010110',
 '202307010111',
 '202307010112']

In [77]:
from urllib.request import urlopen

url = "https://db.netkeiba.com/race/202306050811/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()

In [ ]:
import pandas as pd

pd.read_html(html)[0]

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
0,1,3,5,ドウデュース,牡4,58,武豊,2:30.9,NaN,5.2,2,506(+2),[西] 友道康夫
1,2,8,16,スターズオンアース,牝4,56,ルメール,2:31.0,1/2,8.6,7,494(-2),[東] 高柳瑞樹
2,3,2,4,タイトルホルダー,牡5,58,横山和生,2:31.2,1,8.3,6,474(-2),[東] 栗田徹
3,4,5,10,ジャスティンパレス,牡4,58,横山武史,2:31.2,アタマ,3.6,1,474(+6),[西] 杉山晴紀
4,5,1,2,シャフリヤール,牡5,58,松山弘平,2:31.2,クビ,44.6,8,454(0),[西] 藤原英昭
5,6,7,13,タスティエーラ,牡3,56,ムーア,2:31.5,1.1/2,7.1,5,498(+18),[東] 堀宣行
6,7,6,12,ウインマリリン,牝6,56,モリス,2:31.6,3/4,171.9,14,476(0),[東] 手塚貴久
7,8,1,1,ソールオリエンス,牡3,56,川田将雅,2:31.6,アタマ,6.5,4,466(+2),[東] 手塚貴久
8,9,6,11,ハーパー,牝3,54,岩田望来,2:31.6,クビ,93.2,11,474(-10),[西] 友道康夫
9,10,2,3,ホウオウエミーズ,牝6,56,田辺裕信,2:31.7,クビ,193.2,16,452(+2),[東] 池上昌和


In [80]:
import pickle

with open("race_id_list.pickle", "rb") as f:
    race_id_list = pickle.load(f)

In [82]:
len(race_id_list)

3456

In [83]:
for race_id in race_id_list:
    url = f"https://db.netkeiba.com/race/{race_id}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

    req = Request(url, headers=headers)
    html = urlopen(req).read()
    
    pd.read_html(html)[0]

In [84]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xa5\xab\xa5\xa6\xa5\xf3\xa5\xc8\xa5\xc0\xa5\xa6\xa5\xf3S\xa1\xc32023\xc7\xaf12\xb7\xee28\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa

拡張性のためにHTMLを保存する.

In [89]:
with open("../data/html/202306050811.bin", "wb") as f:
    f.write(html)

In [4]:
from pathlib import Path # windowsとmacによる環境の違いを防ぐためにpathlibを使用

HTML_DIR = Path("..", "data", "html")

In [95]:
with open(HTML_DIR / "202306050811.bin", "wb") as f:
    f.write(html)

In [96]:
import time

for race_id in race_id_list[:10]:
    url = f"https://db.netkeiba.com/race/{race_id}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
    req = Request(url, headers=headers)
    html = urlopen(req).read()
    time.sleep(1)
    pd.read_html(html)[0]
    
    with open(HTML_DIR / f"{race_id}.bin", "wb") as f:
        f.write(html)

In [135]:
from tqdm.notebook import tqdm
def scrape_html_race(race_id_list: list[str], save_dir: Path) -> list[Path]:
    html_path_list = []
    for race_id in tqdm(race_id_list):
        filepath = save_dir / f"{race_id}.bin" # 外で定義した変数を関数内で直接使用しないで, 引数に渡すことを心掛ける.
        # binファイルがすでに存在する場合はスキップする.
        if filepath.is_file():
            print(f"skipped: {race_id}")
        else:
            url = f"https://db.netkeiba.com/race/{race_id}"
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
            req = Request(url, headers=headers)
            html = urlopen(req).read()
            time.sleep(1)
            pd.read_html(html)[0]
            with open(filepath, "wb") as f:
                f.write(html)
            html_path_list.append(filepath)
    return html_path_list

In [140]:
html_path_list = scrape_html_race(race_id_list[:14], save_dir=HTML_DIR)

  0%|          | 0/14 [00:00<?, ?it/s]

skipped: 202306010101
skipped: 202306010102
skipped: 202306010103
skipped: 202306010104
skipped: 202306010105
skipped: 202306010106
skipped: 202306010107
skipped: 202306010108
skipped: 202306010109
skipped: 202306010110
skipped: 202306010111
skipped: 202306010112
skipped: 202307010101
skipped: 202307010102


In [141]:
html_path_list # 新しくスクレイピングしたもののみが帰ってくる.

[]

In [117]:
html_path_list[0].is_file()

True

In [120]:
(HTML_DIR / "aaaa.bin").is_file() # is_fileメソッドで存在するファイルをチェックできる.

False

In [6]:
HTML_RACE_DIR = HTML_DIR / "race"
HTML_RACE_DIR

WindowsPath('../data/html/race')

In [7]:
html_paths_race = list(HTML_RACE_DIR.glob("*")) # glob"*"で全てのPathを取得.
len(html_paths_race)

3456

In [44]:
dfs = {}
for html_path in html_paths_race:
    with open(html_path, "rb") as f:
       race_id = html_path.stem # pathからidのみを取り出せる.
       html = f.read()
       df = pd.read_html(html)[0]
       df.index = [race_id] * len(df) # 難しい：indexを1レースの行数分指定する必要があるので * len(df)
       dfs[race_id] = df
concat_df = pd.concat(dfs.values()) # 複数のテーブルを結合できる.
concat_df.index.name = "race_id"

In [45]:
concat_df

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202305010101,1,7,14,メイデンタワー,牝3,54,横山武史,1:26.1,NaN,1.6,1,444(-4),[東] 稲垣幸雄
202305010101,2,5,10,レディアス,牝3,54,津村明秀,1:26.8,4,6.0,3,424(-4),[東] 鈴木慎太
202305010101,3,5,9,アンジュール,牝3,54,嶋田純次,1:27.1,2,92.7,10,418(0),[東] 佐藤吉勝
202305010101,4,8,15,マテリアルガール,牝3,54,内田博幸,1:27.3,3/4,13.6,4,490(+14),[東] 西田雄一
202305010101,5,2,3,オールマキシマム,牝3,52,小林脩斗,1:27.9,3.1/2,5.8,2,450(+2),[東] 奥平雅士
202305010101,6,4,8,スピリチュアル,牝3,54,武藤雅,1:28.0,1/2,25.7,7,444(-6),[東] 武藤善則
202305010101,7,6,11,マラマプア,牝3,53,永野猛蔵,1:28.1,1,15.2,5,482(+2),[東] 伊藤圭三
202305010101,8,2,4,フジジュンフェイス,牝3,54,杉原誠人,1:28.5,2.1/2,108.1,11,442(-4),[東] 伊藤大士
202305010101,9,4,7,テイエムガーネット,牝3,54,武士沢友,1:28.6,1/2,22.6,6,468(+4),[東] 石栗龍彦


In [19]:
from tqdm.notebook import tqdm
def create_resuls(html_path_list: list[Path]) -> pd.DataFrame:
    dfs = {}
    for html_path in tqdm(html_paths_race):
        with open(html_path, "rb") as f:
            race_id = html_path.stem # pathからidのみを取り出せる.
            html = f.read()
            df = pd.read_html(html)[0]
            df.index = [race_id] * len(df) # 難しい：indexを1レースの行数分指定する必要があるので * len(df)
            dfs[race_id] = df
    concat_df = pd.concat(dfs.values()) # 複数のテーブルを結合できる.
    concat_df.index.name = "race_id"
    return concat_df

In [20]:
results = create_resuls(html_path_list=html_paths_race)

  0%|          | 0/3456 [00:00<?, ?it/s]

In [21]:
results

,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師
race_id,,,,,,,,,,,,,
202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN,1.2,1.0,452(-4),[東] 鈴木伸尋
202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ,4.1,2.0,454(+2),[東] 杉浦宏昭
202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2,59.9,6.0,438(-6),[西] 羽月友彦
202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2,16.6,3.0,450(+2),[西] 武幸四郎
202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2,23.9,5.0,434(-10),[西] 今野貞一
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4,62.8,11.0,394(+2),[西] 松永幹夫
202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2,125.6,12.0,470(-4),[西] 安田隆行
202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4,9.8,5.0,496(+8),[西] 森田直行


# 今回のテーマ
1. 馬の過去成績テーブルの作成
2. Pythonファイルによるファイルの書き込み・読み取り

In [3]:
url = "https://db.netkeiba.com/horse/2020103575"

In [7]:
from urllib.request import urlopen

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()

In [8]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/reset.css?20160421" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/common.css?20210819" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/db_detail.css?20180621" rel="stylesheet" type="text/css" media="all" />\n<link href="https://cdn.netkeiba.com/img.db/common/css/horse_detail.css?20241217" rel="stylesheet" type="text/css" media="all" />\n<link href="

In [13]:
import pandas as pd

pd.read_html(html)[2]

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,...,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2024/12/22,5中山8,晴,8,冬至特別(2勝クラス),NaN,16,1,2,99.0,...,1.4,**,11-11,33.6-38.0,38.4,508(+12),NaN,NaN,ロードフロンティア,NaN
1,2024/08/31,2中京7,小雨,10,鳥羽特別(2勝クラス),NaN,15,3,5,32.6,...,4.6,**,10-10,34.7-36.8,40.5,496(-8),NaN,NaN,カズゴルティス,NaN
2,2024/07/27,2新潟1,曇,12,出雲崎特別(2勝クラス),NaN,15,6,10,8.0,...,0.9,**,4-4,34.2-36.7,37.3,504(+10),NaN,NaN,エランティス,NaN
3,2024/05/19,1新潟8,晴,10,火打山特別(2勝クラス),NaN,15,5,9,13.0,...,0.6,**,6-6,33.6-37.5,37.1,494(-6),NaN,NaN,シラキヌ,230.0
4,2024/05/05,1新潟4,晴,12,4歳以上1勝クラス,NaN,13,4,4,4.5,...,-0.2,**,7-7,35.2-37.6,36.9,500(+2),NaN,NaN,(セキテイオー),800.0
5,2024/02/25,1阪神2,小雨,12,4歳以上1勝クラス,NaN,15,7,13,8.6,...,0.9,**,8-10,34.6-36.4,36.3,498(0),NaN,NaN,サクハル,NaN
6,2023/12/16,5中山5,晴,12,3歳以上1勝クラス,NaN,16,4,7,4.1,...,1.4,**,2-3,34.7-36.5,37.7,498(+4),NaN,NaN,ヤングワールド,NaN
7,2023/09/24,4中山7,晴,8,3歳以上1勝クラス,NaN,16,1,2,2.9,...,1.2,**,4-4,33.2-38.2,38.9,494(-2),NaN,NaN,トモジャミ,80.0
8,2023/08/12,3新潟1,晴,12,3歳以上1勝クラス,NaN,15,8,15,5.0,...,0.6,**,3-3,34.6-36.5,36.7,496(-4),NaN,NaN,キュートヘスティア,320.0
9,2023/07/22,2福島7,晴,12,3歳以上1勝クラス,NaN,16,8,15,8.5,...,0.9,**,10-8,30.8-37.5,37.2,500(+8),NaN,NaN,シアージスト,120.0


In [ ]:
from urllib.request import urlopen

url = "https://db.netkeiba.com/race/202306050811/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}

req = Request(url, headers=headers)
html = urlopen(req).read()

In [44]:
html

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ja" xml:lang="ja" id="html">\n<head>\n\n<title>\xcd\xad\xc7\xcf\xb5\xad\xc7\xb0\xa1\xc32023\xc7\xaf12\xb7\xee24\xc6\xfc | \xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9 - netkeiba</title>\n\n\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n<meta http-equiv="content-language" content="ja" />\n<meta http-equiv="content-type" content="text/html; charset=euc-jp" />\n<meta http-equiv="content-script-type" content="text/javascript" />\n<meta http-equiv="content-style-type" content="text/css" />\n<meta name="description" content="netkeiba\xa4\xac\xb8\xd8\xa4\xeb\xb9\xf1\xc6\xe2\xba\xc7\xc2\xe7\xb5\xe9\xa4\xce\xb6\xa5\xc7\xcf\xa5\xc7\xa1\xbc\xa5\xbf\xa5\xd9\xa1\xbc\xa5\xb9\xa4\xc7\xa4\xb9\xa1\xa350\xcb\xfc\xc6\xac\xb0\xca\xbe\xe5\xa4\xce\xb6\xa5\xc1\xf6\xc7\xcf\xa1\xa2\xb5\xb3\xbc\xea\xa1

In [45]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "lxml")

In [46]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html id="html" lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>有馬記念｜2023年12月24日 | 競馬データベース - netkeiba</title>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="ja" http-equiv="content-language"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎手・調教師・馬主・生産者の全データがご覧いただけます。" name="description"/>
<meta content="競馬情報,競走馬,騎手,レース,調教師,馬主,検索,データベース,JRA,netkeiba,ネット競馬" name="keywords"/>
<!-- ogp用 -->
<meta content="netkeiba" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="競馬データベース | 競走馬・騎手など情報満載 - netkeiba" property="og:title"/>
<meta content="netkeibaが誇る国内最大級の競馬データベースです。50万頭以上の競走馬、騎

In [54]:
table = soup.find("table", class_="race_table_01 nk_tb_common")

In [56]:
table.find_all("a", href="/horse/2019105283/")

[<a href="/horse/2019105283/" id="umalink_202306050811" title="ドウデュース">ドウデュース</a>]

In [61]:
import re

# /horse/から始まるhref属性を持つaタグの一覧を取得する.
a_list = table.find_all("a", href=re.compile(r"^/horse/")) # /horse/を含むhrefに絞る（^は前方一致）

In [63]:
a = a_list[0]
a

<a href="/horse/2019105283/" id="umalink_202306050811" title="ドウデュース">ドウデュース</a>

In [64]:
a["href"]

'/horse/2019105283/'

In [69]:
re.findall(r"\d{10}", a["href"])[0]

'2019105283'

In [71]:
horse_id_list = []
for a in a_list:
    horse_id = re.findall(r"\d{10}", a["href"])[0]
    horse_id_list.append(horse_id)

In [72]:
len(horse_id_list)

16

In [6]:
import create_rawdf
from pathlib import Path
%load_ext autoreload

In [7]:
HTML_RACE_DIR = Path("..", "data", "html", "race")

In [8]:
html_paths_race = list(HTML_RACE_DIR.glob("*.bin"))

In [9]:
results = create_rawdf.create_resuls(html_path_list=html_paths_race[:10])

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
results.isnull().sum()

着順             0
枠番             0
馬番             0
馬名             0
性齢             0
斤量             0
騎手             0
タイム            0
着差            10
単勝             0
人気             0
馬体重            0
調教師            0
horse_id       0
jockey_id      0
trainer_id     0
owner_id       0
dtype: int64

In [11]:
results.columns = results.columns.str.replace(" ","")

In [12]:
save_dir = Path("..", "data", "rawdf")
save_dir.mkdir()

FileExistsError: [WinError 183] 既に存在するファイルを作成することはできません。: '..\\data\\rawdf'

In [13]:
results.to_csv(save_dir / "results.csv", sep="\t") # sep="\t"：tabで区切られたcsv形式

In [14]:
horse_id_list = results["horse_id"].unique() # unique()は重複を無くす.
horse_id_list

array(['2021101429', '2021105872', '2021106854', '2021105553',
       '2021100648', '2021100159', '2021100265', '2021103762',
       '2020106547', '2020100879', '2020100391', '2020104346',
       '2020102908', '2020100551', '2020105380', '2020104250',
       '2020104087', '2020106395', '2020107102', '2020101946',
       '2020104215', '2020103939', '2020102347', '2020105632',
       '2020103126', '2020100863', '2020105061', '2020101252',
       '2020101681', '2020102875', '2020105472', '2020106970',
       '2020100036', '2020101849', '2020105354', '2020103018',
       '2020105782', '2020102563', '2020103346', '2020105792',
       '2020105059', '2020104015', '2020105587', '2020103104',
       '2020106011', '2020104556', '2020106487', '2021106689',
       '2021103728', '2021104889', '2021101661', '2021103385',
       '2021102150', '2021107255', '2021102855', '2021102825',
       '2021101598', '2020105073', '2020103660', '2020102579',
       '2020104281', '2020101146', '2020100175', '20201

In [15]:
len(horse_id_list)

118

In [32]:
import scraping

html_paths_horse = scraping.scrape_html_horse_gpt(horse_id_list=horse_id_list)

  0%|          | 0/118 [00:00<?, ?it/s]

skipped: 2021101429
skipped: 2021105872
skipped: 2021106854
skipped: 2021105553
skipped: 2021100648
skipped: 2021100159
skipped: 2021100265
skipped: 2021103762
skipped: 2020106547
skipped: 2020100879
skipped: 2020100391
skipped: 2020104346
skipped: 2020102908
skipped: 2020100551
skipped: 2020105380
skipped: 2020104250
skipped: 2020104087
Attempt 1 failed with error: HTTP Error 400: Bad Request
Attempt 2 failed with error: HTTP Error 400: Bad Request
Attempt 3 failed with error: HTTP Error 400: Bad Request
Attempt 4 failed with error: HTTP Error 400: Bad Request
Attempt 5 failed with error: HTTP Error 400: Bad Request
Attempt 6 failed with error: HTTP Error 400: Bad Request
Attempt 7 failed with error: HTTP Error 400: Bad Request
Attempt 8 failed with error: HTTP Error 400: Bad Request
Attempt 9 failed with error: HTTP Error 400: Bad Request
Attempt 10 failed with error: HTTP Error 400: Bad Request
Attempt 11 failed with error: HTTP Error 400: Bad Request
Attempt 12 failed with error: H

KeyboardInterrupt: 

In [17]:
html_paths_horse

NameError: name 'html_paths_horse' is not defined

In [20]:
%autoreload

In [24]:
HTML_HORSE_DIR = Path("..", "data", "html", "horse")

In [25]:
html_paths_horse = list(HTML_HORSE_DIR.glob("*.bin"))

In [26]:
horse_results = create_rawdf.create_horse_resuls(html_path_list=html_paths_horse)

  0%|          | 0/17 [00:00<?, ?it/s]

In [28]:
horse_results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
horse_id,,,,,,,,,,,,,,,,,,,,,
2020100391,2023/08/05,1札幌5,曇,2,3歳未勝利,NaN,12,6,8,9.2,...,2.2,**,9-10,34.5-34.8,36.1,456(+10),NaN,NaN,ダラムキャッスル,NaN
2020100391,2023/07/22,1札幌1,晴,2,3歳未勝利,NaN,12,3,3,99.0,...,0.9,**,6-7,34.6-35.6,35.5,446(-4),NaN,NaN,クリーデンス,140.0
2020100391,2023/05/21,2東京10,曇,3,3歳未勝利,NaN,16,5,10,22.2,...,0.9,**,13-12,28.9-37.9,37.1,450(0),NaN,NaN,キングダラス,NaN
2020100391,2023/04/30,2東京4,曇,2,3歳未勝利,NaN,16,2,4,30.5,...,0.8,**,8-8,35.9-37.4,37.4,450(-4),NaN,NaN,マスグラバイト,55.0
2020100391,2023/04/09,3中山6,晴,3,3歳未勝利,NaN,16,5,9,64.2,...,1.7,**,10-10,33.4-38.1,38.4,454(-10),NaN,NaN,リラックス,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021106854,2024/02/03,1小倉7,小雨,9,かささぎ賞(1勝クラス),NaN,12,8,12,73.0,...,1.4,**,4-3,33.4-35.3,36.3,434(-4),NaN,NaN,ペアポルックス,NaN
2021106854,2023/12/02,5阪神1,晴,9,さざんか賞(1勝クラス),NaN,10,3,3,79.9,...,1.2,**,4-4,32.6-35.9,36.0,438(0),NaN,NaN,ジャスパーノワール,NaN
2021106854,2023/11/11,3福島3,曇,3,2歳未勝利,NaN,16,3,5,8.1,...,-0.1,**,2-2,34.4-36.4,36.3,438(0),NaN,NaN,(シャインヴィーナス),550.0
